# Create HiRID External validation dataset (Experiment 1)

In [ ]:
#Import all necessary modules

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#Connect to HiRID database

import psycopg2
from psycopg2 import Error

conn = psycopg2.connect(user="mimicuser",
                                  password="knowlabMIMIC",
                                  host="172.17.0.1",
                                  port="5433",
                                  database="HiRID")

cur = conn.cursor()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### 1) Filtering for all instances with drug administration (Adrenaline/Noradrenaline)

HiRID documentation contains the variableids for all drug variables:<br>
https://hirid.intensivecare.ai/structure-of-the-published-data

- **Adrenaline** (continuously administered): 1000649, 1000650
- **Noradrenaline** (continuously administered): 1000656, 1000657

In [ ]:
#Connect to HiRID pharma_records table
##filtering for only continuous Adrenaline/ Noradrenaline fields
##pharmaids for continuous Adrenaline/Noradrenaline fields found in HiRID documentation - https://hirid.intensivecare.ai/structure-of-the-published-data

#import all records from HiRID Pharma table containing continuous Adrenaline/Noradrenaline
drugs_df = pd.read_sql_query("SELECT * FROM hirid.pharma_records \
                         WHERE pharmaid IN (1000649, 1000650, 1000656, 1000657) \
                         ORDER BY patientid, pharmaid, givenat", conn)


#categorise drugs_df 'givenat' column to closest 15min timepoint (needed for later analysis)
drugs_df['timepoint'] = drugs_df['givenat'].dt.round('15min') 

print(drugs_df.shape)
drugs_df.head()

##1,458,539 records
##NOTE: includes rows where given_dose=0

In [ ]:
#Filter for records containing Adrenaline/Noradrenaline readings with non-zero values 

drugs_nonzero = drugs_df[drugs_df.givendose != 0] #drop all rows where givendose=0.0

print(drugs_nonzero.shape)
drugs_nonzero

##1,313,057 records

In [ ]:
#check datatypes in drugs_df table

drugs_df.dtypes

In [ ]:
#if necessary - convert givenat to datetime format

drugs_df['givenat'] =  pd.to_datetime(drugs_df['givenat'], format='%Y-%m-%d %H:%M:%S.%f')
drugs_df.dtypes

In [ ]:
#show count of each drug field

drugs_df['pharmaid'].value_counts()

In [ ]:
#show all route values

drugs_df['route'].unique()

##only continuous administration - as required 

In [ ]:
#number of unique ICU admissions
##note: patientid corresponds to ICU admissions (not patient identifier)

pat_drugs = list(drugs_df['patientid'].unique())
len(pat_drugs)

#10,665 unique ICU admissions

In [ ]:
#Check number of unique instances (patientid + givenat)

agg = drugs_df.groupby(['patientid','givenat'])['pharmaid'].nunique()
agg = pd.DataFrame(data=agg, index=None)
agg = agg.reset_index()
print(agg.shape)
agg.head(10)

##1,430,607 unique instances 

In [ ]:
#Filter for 15min time intervals (per patientid) containing (non-zero) readings for Adrenaline/Noradrenaline

agg_drugs = drugs_nonzero.groupby([pd.Grouper(key='givenat', freq='15Min'), 'patientid']).pharmaid.unique()
agg_drugs = pd.DataFrame(data=agg_drugs)
agg_drugs = agg_drugs.reset_index()
agg_drugs = agg_drugs.sort_values(by=['patientid','givenat'], ascending=[True,True])

print(agg_drugs.shape)
agg_drugs.head(15)

#818,425 unique instances (with non-zero values for Adrenaline/Noradrenaline) 

In [ ]:
#check datatypes - givenat should be in datetime format

agg_drugs.dtypes

In [ ]:
#check unique ICU admissions

agg_drugs['patientid'].nunique()

##10,634 unique ICU admissions that have non-zero values for Adrenaline/Noradrenaline

### 2) Calculate drug rate for instances with Adrenaline/Noradrenaline drug readings (mg/hr)

In [ ]:
#Order drugs_df table by patientid, pharmaid, givenat

drugs_df.sort_values(by=['patientid','pharmaid','givenat'])

print(drugs_df.shape)
drugs_df.head()

##1,458,539 records

In [ ]:
#Confirm givenat is in datetime format

drugs_df.dtypes

In [ ]:
#Extract relevant columns from drugs_df

drugs_mod = drugs_df.copy(deep=True)
drugs_mod = drugs_mod[['patientid', 'pharmaid', 'givenat', 'timepoint', 'givendose','cumulativedose','doseunit']]

print(drugs_mod.shape)
drugs_mod.head()

##1,458,539 records

In [ ]:
#Check drug units

drugs_mod['doseunit'].unique()

##unit = micrograms

In [ ]:
#Get min_time for each (patientid, pharmaid, timepoint)
##min_time = minimum (earliest) drug administration time within given timepoint

temp = drugs_mod.groupby(['patientid','pharmaid', 'timepoint'])
min_time = temp.agg(min_time=('givenat', np.min))
min_time = min_time.reset_index()
min_time

In [ ]:
#Get cumulative dose for min_time per patientid in drugs_mod  

merge = pd.merge(min_time, drugs_mod, left_on=['patientid','pharmaid','timepoint','min_time'], right_on=['patientid','pharmaid','timepoint','givenat'])
min_cumval = merge[['patientid', 'pharmaid','timepoint','min_time','cumulativedose']]
min_cumval.rename({'cumulativedose': 'cumulativedose(min)'}, axis=1, inplace=True)
min_cumval

In [ ]:
#Get max_time for each (patientid, pharmaid, timepoint)
##max_time = maximum (latest) drug administration time within given timepoint

temp = drugs_mod.groupby(['patientid','pharmaid','timepoint'])
max_time = temp.agg(max_time=('givenat', np.max))
max_time = max_time.reset_index()
max_time

In [ ]:
#Get cumulative dose for max_time per patientid in drugs_mod  

merge = pd.merge(max_time, drugs_mod, left_on=['patientid','pharmaid','timepoint','max_time'], right_on=['patientid','pharmaid','timepoint','givenat'])
max_cumval = merge[['patientid', 'pharmaid','timepoint','max_time','cumulativedose']]
max_cumval.rename({'cumulativedose': 'cumulativedose(max)'}, axis=1, inplace=True)
max_cumval

In [ ]:
#Calculate drugs rate

drugs_rate = pd.merge(min_cumval, max_cumval, left_on=['patientid','pharmaid','timepoint'], right_on=['patientid','pharmaid','timepoint'])

drugs_rate['time_diff'] = drugs_rate['max_time']-drugs_rate['min_time']
drugs_rate['time_diff(mins)'] = (drugs_rate['time_diff'] / np.timedelta64(1, 'h'))*60
drugs_rate['cumulativedose_diff'] = drugs_rate['cumulativedose(max)']-drugs_rate['cumulativedose(min)']

drugs_rate['rate(microg/min)']= drugs_rate['cumulativedose_diff']/drugs_rate['time_diff(mins)']
drugs_rate

In [ ]:
#Covert rate units from microgrames/min to milligrams/hr

conversion = 0.060000071999942

drugs_rate['rate(millig/hr)']=drugs_rate['rate(microg/min)']*conversion
drugs_rate

In [ ]:
#Filter for acceptable rate values 

drugs_rate = drugs_rate[drugs_rate['rate(millig/hr)']<10]
drugs_rate = drugs_rate[drugs_rate['rate(millig/hr)']>0]

print(drugs_rate.shape)
drugs_rate

##272,932 records

In [ ]:
#Check min and max drug rate

max_rate = drugs_rate['rate(millig/hr)'].max()
min_rate = drugs_rate['rate(millig/hr)'].min()
print("Lowest drug rate:", min_rate)
print("Highest drug rate:", max_rate)

In [ ]:
#Rename pharmaid to drug name 

drugs_rate['pharmaid'] = drugs_rate['pharmaid'].replace({ 1000649 : "Adrenaline", 1000650 : "Adrenaline", 
                                                         1000656: "Noradrenaline", 1000657 : "Noradrenaline"})
drugs_rate.head()

In [ ]:
#Extract relevant columns from drugs_rate

drugs_rate = drugs_rate[['patientid','pharmaid','timepoint','rate(millig/hr)']]

print(drugs_rate.shape)
drugs_rate.head()

In [ ]:
#Reformat drugs_rate DF to structure similar to training data 

drugs_rate = drugs_rate.groupby(['patientid', 'timepoint', 'pharmaid'])['rate(millig/hr)'].sum().unstack('pharmaid')

drugs_rate = drugs_rate.reset_index()
drugs_rate = drugs_rate.rename_axis(None, axis=1)

drugs_rate['Adrenaline'] = round(drugs_rate['Adrenaline'],2)
drugs_rate['Noradrenaline'] = round(drugs_rate['Noradrenaline'],2)

print(drugs_rate.shape)
drugs_rate

#268,461 records

### 3) Find instances with physiological parameter readings (FiO2/SpO2/HR/MAP)

HiRID documentation contains the variableids for all physiological parameters:<br>
https://hirid.intensivecare.ai/structure-of-the-published-data

- **FiO2**: 2010
- **SpO2**: 4000, 8280
- **Heart Rate**: 200
- **Mean Arterial Pressure**: 110, 610

In [ ]:
#Connect to observations_table_1

obs_tab1 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_1 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab1['row_idx']=obs_tab1.index #create new index column

obs_tab1.to_csv('obs_tab1.csv')

print(obs_tab1.shape)
obs_tab1.head()

#number of records = 19,492,758

In [ ]:
#Connect to observations_table_2

obs_tab2 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_2 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab2['row_idx']=obs_tab2.index #create new index column

obs_tab2.to_csv('obs_tab2.csv')

print(obs_tab2.shape)
obs_tab2.head()

#number of records = 19,373,029

In [ ]:
#Connect to observations_table_3

obs_tab3 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_3 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab3['row_idx']=obs_tab3.index #create new index column

obs_tab3.to_csv('obs_tab3.csv')

print(obs_tab3.shape)
obs_tab3.head()

#number of records = 19,922,456

In [ ]:
#Connect to observations_table_4

obs_tab4 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_4 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab4['row_idx']=obs_tab4.index #create new index column

obs_tab4.to_csv('obs_tab4.csv')

print(obs_tab4.shape)
obs_tab4.head()

#number of records = 19,576,004

In [ ]:
#Connect to observations_table_5

obs_tab5 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_5 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab5['row_idx']=obs_tab5.index #create new index column

obs_tab5.to_csv('obs_tab5.csv')

print(obs_tab5.shape)
obs_tab5.head()

#number of records = 19,475,848

In [ ]:
#Connect to observations_table_6

obs_tab6 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_6 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab6['row_idx']=obs_tab6.index #create new index column

obs_tab6.to_csv('obs_tab6.csv')

print(obs_tab6.shape)
obs_tab6.head()

#number of records = 19,612,923

In [ ]:
#Connect to observations_table_7

obs_tab7 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_7 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab7['row_idx']=obs_tab7.index #create new index column

obs_tab7.to_csv('obs_tab7.csv')

print(obs_tab7.shape)
obs_tab7.head()

## number of records = 19,818,593

In [ ]:
#Connect to observations_table_8

obs_tab8 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_8 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab8['row_idx']=obs_tab8.index #create new index column

obs_tab8.to_csv('obs_tab8.csv')

print(obs_tab8.shape)
obs_tab8.head()

#number of records = 19,449,724

In [ ]:
#Connect to observations_table_9

obs_tab9 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_9 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab9['row_idx']=obs_tab9.index #create new index column

obs_tab9.to_csv('obs_tab9.csv')

print(obs_tab9.shape)
obs_tab9.head()

#number of records = 19,922,926

In [ ]:
#Connect to observations_table_10

obs_tab10 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_10 \
                                    WHERE variableid IN (200, 110, 610, 4000, 8280, 2010) \
                                    ORDER BY patientid, datetime ", conn)

obs_tab10['row_idx']=obs_tab10.index #create new index column

obs_tab10.to_csv('obs_tab10.csv')

print(obs_tab10.shape)
obs_tab10.head()

#number of records = 18,538,580

In [ ]:
print('Number of records observation_table_1:', len(obs_tab1))
print('Number of records observation_table_2:', len(obs_tab2))
print('Number of records observation_table_3:', len(obs_tab3))
print('Number of records observation_table_4:', len(obs_tab4))
print('Number of records observation_table_5:', len(obs_tab5))
print('Number of records observation_table_6:', len(obs_tab6))
print('Number of records observation_table_7:', len(obs_tab7))
print('Number of records observation_table_8:', len(obs_tab8))
print('Number of records observation_table_9:', len(obs_tab9))
print('Number of records observation_table_10:', len(obs_tab10))

In [ ]:
#Concatenate above observation tables datasets (1-10)

frames = [obs_tab1, obs_tab2, obs_tab3, obs_tab4, obs_tab5, obs_tab6, obs_tab7, obs_tab8, obs_tab9, obs_tab10]

phys_params = pd.concat(frames)

phys_params['timepoint'] = phys_params['datetime'].dt.round('15min') #timepoint col needed in later manipulationn

phys_params.to_csv('phys_params.csv')

print(phys_params.shape)
phys_params

##195,182,841 records

### 4) Find instances with 15min intervals containing readings for all 4 physiological parameters

In [ ]:
#Using phys_params
#List unique physiological variableids within 15min time intervals, per patientid 
##datetime needs to be in correct format for query to run

agg_var_full = phys_params.groupby([pd.Grouper(key='datetime', freq='15Min'), 'patientid']).variableid.unique()
agg_var_full = pd.DataFrame(data=agg_var_full)
agg_var_full = agg_var_full.reset_index()
agg_var_full = agg_var_full.sort_values(by=['patientid','datetime'], ascending=[True,True])

agg_var_full.to_csv('agg_var_full.csv')

print(agg_var_full.shape)
agg_var_full.head(5)

##7,204,889 records

In [ ]:
#Filter agg_var_full for all instances that contain a reading for FiO2

#FiO2
df_mod = agg_var_full[agg_var_full.variableid.astype(str).str.contains('2010')]
print(df_mod.shape)
df_mod

##3,040,811 records

In [ ]:
#Filter df_mod for all instances that contain a reading for SpO2

df_mod = df_mod[df_mod.variableid.astype(str).str.contains('4000|8280')]
print(df_mod.shape)
df_mod

##3,021,057 records

In [ ]:
#Filter df_mod for all instances that contain a reading for HR

df_mod = df_mod[df_mod.variableid.astype(str).str.contains('200')]
print(df_mod.shape)
df_mod

##3,020,405

In [ ]:
#Filter df_mod for all instances that contain a reading for MAP

df_mod = df_mod[df_mod.variableid.astype(str).str.contains('110|610')]

df_mod.to_csv('df_mod.csv')

print(df_mod.shape)
df_mod

##3,004,013
##There are 3,004,013 unique instances which contain readings for all 4 physiological parameters
##This is our pool of relevant (patientid + timepoint) combinations

In [ ]:
#Check unique ICU admissions

df_mod['patientid'].nunique()

##20,289 unique ICU admissions within above pool of relevant instances

In [ ]:
df_mod.groupby(['datetime', 'patientid']).ngroups

#confirms there are 3,004,013 unique combinations of (15min time interval + patientid) that have readings for 4 phys params

### 5) Find instances with readings for Adrenaline/Noradrenaline AND all 4 physiological parameters

In [ ]:
#Merge df_mod and agg_drugs
##Pool of relevant instances (unique timepoint + patientid)

agg_pool = pd.merge(df_mod, agg_drugs,  how='left', left_on=['datetime','patientid'], right_on = ['givenat','patientid'])
agg_pool = agg_pool.drop(['givenat'],axis=1)
agg_pool.rename({'datetime': 'timepoint'}, axis=1, inplace=True)

agg_pool['timepoint'] =  pd.to_datetime(agg_pool['timepoint'], format='%Y-%m-%d %H:%M:%S')

agg_pool.to_csv('agg_pool.csv')

print(agg_pool.shape)
agg_pool

#3,004,013 instances that have 4+ readings for phyiological parameters 

In [ ]:
#if necessary - convert timepoint to datetime format
agg_pool['timepoint'] =  pd.to_datetime(agg_pool['timepoint'], format='%Y-%m-%d %H:%M:%S')
agg_pool.dtypes

In [ ]:
#Check how many instances (unique patientid + timepoint) are common between df_mod and agg_drugs 

agg_merge = pd.merge(agg_drugs, df_mod,  how='left', left_on=['givenat','patientid'], right_on = ['datetime','patientid'])
agg_merge = agg_merge.drop(['datetime'],axis=1)
agg_merge.rename({'givenat': 'timepoint'}, axis=1, inplace=True)

agg_merge = agg_merge[agg_merge['variableid'].notna()]

print(agg_merge.shape)
agg_merge.head()

#610,917 instances with non-zero readings for Adrenaline/Noradrenaline AND all 4 physiological parameters

### 6) For (patientid + timepoint) present in agg_pool, find all instances of physiological parameter readings

In [ ]:
#Filter phys_params for only relevant (patientid + timepoint) combinations (ie. those present in agg_pool dataframe)

params_mod = pd.merge(phys_params, agg_pool,  how='left', left_on=['timepoint','patientid'], right_on = ['timepoint','patientid'])

params_mod = params_mod[params_mod['variableid_y'].notna()]

params_mod.to_csv('params_mod.csv')

print(params_mod.shape)
params_mod.head()

##91,507,817 records

In [ ]:
#Rename variableid_x
params_mod.rename({'variableid_x': 'variable'}, axis=1, inplace=True)
params_mod.dtypes

In [ ]:
#Extract relevant cols from params_comp_mod

params_comp = params_mod[['patientid','timepoint', 'variable', 'value']]

params_comp = params_comp.sort_values(by=['patientid','timepoint','variable'], ascending=[True,True,True])

print(params_comp.shape)
params_comp.head()

##91,507,817 records

In [ ]:
#Replace variable numbers by names

params_comp['variable'] = params_comp['variable'].replace({ 2010 : "FiO2", 4000 : "SpO2", 8280: "SpO2", 200 : "HR", 
                                                    110 : "MAP", 610 : "MAP"})
print(params_comp.shape)
params_comp.head()

##91,507,817 records

In [ ]:
#List all variables per unique (patientid + timepoint)
##Primary Key = (patientid + timepoint + variable)

params_comp = params_comp.groupby(['patientid','timepoint', 'variable'])['value'].apply(list)

params_comp = pd.DataFrame(data=params_comp)
params_comp = params_comp.reset_index()

params_comp.to_csv('params_comp.csv')

print(params_comp.shape)
params_comp

##11,955,825 records

In [ ]:
#check unique ICU admissions in params_comp

params_comp['patientid'].nunique()

##20,286 unique ICU admissions

### 7) Check all variable values are in acceptable range with compatible units

In [ ]:
#Check format of 'value' column

for i, l in enumerate(params_comp["value"]):
    print("list",i,"is",type(l))

In [ ]:
#IF format = String: Change format of 'value'column from string to list (in order to loop through)
##IF format = List: skip this step

def clean_alt_list(list_):
    list_ = list_.replace(', ', '","')
    list_ = list_.replace('[', '["')
    list_ = list_.replace(']', '"]')
    return list_

params_comp["value"]= params_comp["value"].apply(clean_alt_list)
params_comp["value"] = params_comp["value"].apply(eval)

In [ ]:
#Add row identifier column

params_comp['row_idx']=params_comp.index 
print(params_comp.shape)
params_comp.head()

In [ ]:
#Convert value column to array - in order to loop through each list

arr = params_comp.value.apply(lambda x: [float(c) for c in x])
arr

In [ ]:
#Convert value column back to dataframe (and rename)

arr_df = pd.DataFrame(data=arr,index=None)
arr_df.columns = ['val_list']
arr_df['row_idx']=arr_df.index #create new index column
arr_df

In [ ]:
#Merge formatted val_list column with params_comp

params_comp = pd.merge(params_comp, arr_df,  how='left', on=['row_idx'])
params_comp = params_comp.drop(['row_idx','value'],axis=1)
params_comp

##11,955,825 records

In [ ]:
#Check all FiO2 values are in acceptable range

check_fio2 = params_comp.copy(deep=True)

check_fio2 = check_fio2[check_fio2['variable']=='FiO2']
check_fio2['fio2_check'] = check_fio2.val_list.apply(lambda x: [ all(c >= 21 and c <=100 for c in x)])
check_fio2['acceptable_range'] = check_fio2.fio2_check.apply(lambda x: 'Yes' if True in x else 'No')

check_fio2

##2,983,684 records

In [ ]:
len(check_fio2[check_fio2['acceptable_range']=='No'])

#2,439 instances containing FiO2 values outside acceptable range 

In [ ]:
#Create new column FiO2 values that are in acceptable range

check_fio2['inrange_val_list'] = check_fio2.val_list.apply(lambda x: [ c for c in x if 21 <= c <= 100])
check_fio2 = check_fio2.drop(['val_list','fio2_check','acceptable_range'],axis=1)

check_fio2 = check_fio2.dropna()

print(check_fio2.shape)
check_fio2.head()

##2,983,684 records

In [ ]:
#Confirm all FiO2 values are in acceptable range

check_fio2['fio2_check'] = check_fio2.inrange_val_list.apply(lambda x: [ all(c >= 21 and c <=100 for c in x)])
check_fio2['acceptable_range'] = check_fio2.fio2_check.apply(lambda x: 'Yes' if True in x else 'No')

check_fio2['acceptable_range'].unique()

##All FiO2 values are now in acceptable range 

In [ ]:
#Drop unncessary columns

check_fio2 = check_fio2.drop(['fio2_check','acceptable_range'],axis=1)
print(check_fio2.shape)
check_fio2.head()

##2,983,684 records

In [ ]:
#Check all SpO2 values are in acceptable range

check_spo2 = params_comp.copy(deep=True)

check_spo2 = check_spo2[check_spo2['variable']=='SpO2']
check_spo2['spo2_check'] = check_spo2.val_list.apply(lambda x: [ all(c >= 0 and c <=100 for c in x)])
check_spo2['acceptable_range'] = check_spo2.spo2_check.apply(lambda x: 'Yes' if True in x else 'No')

check_spo2

##2,988,937 records

In [ ]:
len(check_spo2[check_spo2['acceptable_range']=='No'])

#14 instances contain SpO2 values outside acceptable ranges 

In [ ]:
#Create new column SpO2 values that are in acceptable range

check_spo2['inrange_val_list'] = check_spo2.val_list.apply(lambda x: [ c for c in x if 0 <= c <= 100])
check_spo2 = check_spo2.drop(['val_list','spo2_check','acceptable_range'],axis=1)

check_spo2 = check_spo2.dropna()

print(check_spo2.shape)
check_spo2.head()

##2,988,937 records

In [ ]:
#Confirm all SpO2 values are in acceptable range

check_spo2['spo2_check'] = check_spo2.inrange_val_list.apply(lambda x: [ all(c >= 0 and c <=100 for c in x)])
check_spo2['acceptable_range'] = check_spo2.spo2_check.apply(lambda x: 'Yes' if True in x else 'No')

check_spo2['acceptable_range'].unique()

##All SpO2 values are now in acceptable range 

In [ ]:
#Drop unncessary columns

check_spo2 = check_spo2.drop(['spo2_check','acceptable_range'],axis=1)
print(check_spo2.shape)
check_spo2.head()

##2,988,937 records

In [ ]:
#Check all MAP values are in acceptable range

check_map = params_comp.copy(deep=True)

check_map = check_map[check_map['variable']=='MAP']
check_map['map_check'] = check_map.val_list.apply(lambda x: [ all(c >= 0 and c <=200 for c in x)])
check_map['acceptable_range'] = check_map.map_check.apply(lambda x: 'Yes' if True in x else 'No')

check_map

##2,989,610 records

In [ ]:
len(check_map[check_map['acceptable_range']=='No'])

#61,820 instances contain an MAP reading outside the acceptable range 

In [ ]:
#Create new column MAP values that are in acceptable range

check_map['inrange_val_list'] = check_map.val_list.apply(lambda x: [ c for c in x if 0 <= c <= 200])
check_map = check_map.drop(['val_list','map_check','acceptable_range'],axis=1)

check_map = check_map.dropna()

print(check_map.shape)
check_map.head()

##2,989,610 records

In [ ]:
#Confirm all MAP values are in acceptable range

check_map['map_check'] = check_map.inrange_val_list.apply(lambda x: [ all(c >= 0 and c <=200 for c in x)])
check_map['acceptable_range'] = check_map.map_check.apply(lambda x: 'Yes' if True in x else 'No')

check_map['acceptable_range'].unique()

##All MAP values are now in acceptable range

In [ ]:
#Drop unncessary columns

check_map = check_map.drop(['map_check','acceptable_range'],axis=1)
print(check_map.shape)
check_map.head()

##2,989,610 records

In [ ]:
#Check all HR values are in acceptable range

check_hr = params_comp.copy(deep=True)

check_hr = check_hr[check_hr['variable']=='HR']
check_hr['hr_check'] = check_hr.val_list.apply(lambda x: [ all(c >= 0 and c <=300 for c in x)])
check_hr['acceptable_range'] = check_hr.hr_check.apply(lambda x: 'Yes' if True in x else 'No')
check_hr

##2,993,594 records

In [ ]:
len(check_hr[check_hr['acceptable_range']=='No'])

#All HR values are in acceptable range 

In [ ]:
#Drop unncessary columns

check_hr = check_hr.drop(['hr_check','acceptable_range'],axis=1)
check_hr.rename({'val_list': 'inrange_val_list'}, axis=1, inplace=True)

print(check_hr.shape)
check_hr.head()

In [ ]:
print(check_fio2.columns)
print(check_spo2.columns)
print(check_map.columns)
print(check_hr.columns)

In [ ]:
#Concatenate check,fio2, check_spo2, check_map check_hr

frames = [check_fio2, check_spo2, check_map, check_hr]

params_comp = pd.concat(frames)

params_comp.sort_values(by=['patientid','timepoint'], ascending=[True,True])

print(params_comp.shape)
params_comp

##11,955,825 records

### 8)  Extract minimum, maximum and average variable values per unique instance

In [ ]:
#Extract minimum and maximum variable values

params_comp['min_val'] = params_comp.inrange_val_list.apply(lambda x: min(x, default=np.nan))
params_comp['max_val'] = params_comp.inrange_val_list.apply(lambda x: max(x, default=np.nan))

print(params_comp.shape)
params_comp.head()

##11,955,825 records

In [ ]:
#Extract average variable value

import statistics

params_comp = params_comp.dropna() #drop all rows containing null values (indicates empty value list)

params_comp['avg_val'] = params_comp.inrange_val_list.apply(lambda x: round(statistics.mean(x),1))

print(params_comp.shape)
params_comp.head()

##11,955,670 records

### 9) Remove instances where % variation is larger than defined cut-off

As advised by our medical advisor, Prof Malcolm Sim, a large variation should not be observed within a such a short time period, and is indicative of a data error. <br>
•	**% variation** = (( max_val – min_val ) / min_val ) x 100


In [ ]:
#Check % variation of variable readings for each instance

params_comp['variation(%)'] = ((params_comp['max_val']-params_comp['min_val'])/params_comp['min_val'])*100

print(params_comp.shape)
params_comp.head() 

##11,955,670 records

In [ ]:
#Remove instances where the % variation of values within each 15-minute timepoint is within is greater than defined cut-off
###Instances to remove - cut-offs advised by X (medical advisor to project)

params_comp = params_comp[((params_comp['variable']=='FiO2') & (params_comp['variation(%)']<=5))|
                          ((params_comp['variable']=='SpO2') & (params_comp['variation(%)']<=10))|
                          ((params_comp['variable']=='MAP') & (params_comp['variation(%)']<=25))|
                          ((params_comp['variable']=='HR') & (params_comp['variation(%)']<=25))]

print(params_comp.shape)
params_comp.head() 

#10,719,315 records

### 10) Check average parameter values match training data units and ranges

In [ ]:
#Extract relevant columns from params_comp
##Reformat dataframe to similar structure as Glasgow training data, using average parameter values 

avg_params = params_comp[['patientid','timepoint','variable','avg_val']]

avg_params = avg_params.groupby(['patientid', 'timepoint', 'variable'])['avg_val'].sum().unstack('variable')

avg_params = avg_params.reset_index()
avg_params = avg_params.rename_axis(None, axis=1)
avg_params = avg_params.dropna() #drop any rows that contain null values 

print(avg_params.shape)
avg_params

##2,022,310 records
##need to convert FiO2 units from % to decFrc (HR, MAP, SpO2 have correct units)

In [ ]:
#Convert FiO2 to decFrc (compatible with Glasgow training data)

avg_params['FiO2']=round((avg_params['FiO2']/100),2)

In [ ]:
##Confirm all average values are within acceptable range

avg_params = avg_params[avg_params['FiO2']>=0.21]
avg_params = avg_params[avg_params['FiO2']<=1.0]

avg_params = avg_params[avg_params['SpO2']>0]
avg_params = avg_params[avg_params['SpO2']<=100]

avg_params = avg_params[avg_params['HR']>0]
avg_params = avg_params[avg_params['HR']<=300]

avg_params = avg_params[avg_params['MAP']>0]
avg_params = avg_params[avg_params['MAP']<=200]

print(avg_params.shape)
avg_params

##2,022,310 records

In [ ]:
#Check number of unique ICU admissions in avg_params

avg_params['patientid'].nunique()

##20,073 unique ICU admissions

### 11) Create HiRID external dataset using Average parameter values 

In [ ]:
print(avg_params.shape)
avg_params.head()

##2,022,310 records

In [ ]:
drugs_rate.head()

In [ ]:
#if necessary - convert timepoint to correct datetime format
drugs_rate['timepoint'] =  pd.to_datetime(drugs_rate['timepoint'], format='%Y-%m-%d %H:%M:%S')
drugs_rate.dtypes

In [ ]:
#Merge avg_params and drugs_rate on (patientid + timepoint)

comp_avg_params = pd.merge(avg_params, drugs_rate, how='left', on=['patientid','timepoint'])
comp_avg_params = comp_avg_params[['patientid','timepoint','Adrenaline','Noradrenaline','FiO2','SpO2','MAP','HR']]

comp_avg_params['rowID']=comp_avg_params.index #create new index column

#move index col to start
col_at_start = ['rowID']
comp_avg_params = comp_avg_params[[c for c in col_at_start if c in comp_avg_params] + [c for c in comp_avg_params if c not in col_at_start]]

#replace null with 0 in drug fields (as blank value indicates value=0)
comp_avg_params['Adrenaline'] = comp_avg_params['Adrenaline'].replace(np.nan, 0)
comp_avg_params['Noradrenaline'] = comp_avg_params['Noradrenaline'].replace(np.nan, 0)

print(comp_avg_params.shape)
comp_avg_params

##2,022,310 records

In [ ]:
#Check number of non-zero drug fields

print(len(comp_avg_params[comp_avg_params['Adrenaline']>0]))
print(len(comp_avg_params[comp_avg_params['Noradrenaline']>0]))

### 12) Find discharge & death times for each unique ICU admission

As the HiRID databse does not contain data for an ICU discharge/death time, the last Heart Rate reading was used as the discharge time (if discharge_status=alive) or death time (if discharge_status=dead). This approach was recommended by the HiRID v1.1.1 database corresponding author, Martin Faltys.

In [ ]:
#Connect to observations_table_1 - find all HR readings

hr_tab1 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_1 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab1['row_idx']=hr_tab1.index #create new index column

print(hr_tab1.shape)
hr_tab1.head()

##6,076,840 records

In [ ]:
hr_tab1_patlist = list(hr_tab1['patientid'].unique())
len(hr_tab1_patlist)

##3386 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab1 -> this is equivalent to discharge/death time 

temp = hr_tab1.groupby('patientid')
dtime_tab1 = temp.agg(max_date=('datetime', np.max))
dtime_tab1 = dtime_tab1.reset_index()
dtime_tab1

#max_date = discharge time/ death time for all patientids in hr_tab1

In [ ]:
#Connect to observations_table_2 - find all HR readings

hr_tab2 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_2 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab2['row_idx']=hr_tab2.index #create new index column

print(hr_tab2.shape)
hr_tab2.head()

##5,943,365 records

In [ ]:
hr_tab2_patlist = list(hr_tab2['patientid'].unique())
len(hr_tab2_patlist)

##3391 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab2

temp = hr_tab2.groupby('patientid')
dtime_tab2 = temp.agg(max_date=('datetime', np.max))
dtime_tab2 = dtime_tab2.reset_index()
dtime_tab2

#max_date = discharge time/ death time for all patientids in hr_tab2

In [ ]:
#Connect to observations_table_3 - find all HR readings

hr_tab3 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_3 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab3['row_idx']=hr_tab3.index #create new index column

print(hr_tab3.shape)
hr_tab3.head()

##6,157,038 records

In [ ]:
hr_tab3_patlist = list(hr_tab3['patientid'].unique())
len(hr_tab3_patlist)

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab3

temp = hr_tab3.groupby('patientid')
dtime_tab3 = temp.agg(max_date=('datetime', np.max))
dtime_tab3 = dtime_tab3.reset_index()
dtime_tab3

#max_date = discharge time/ death time for all patientids in hr_tab3

In [ ]:
#Connect to observations_table_4 - find all HR readings

hr_tab4 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_4 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab4['row_idx']=hr_tab4.index #create new index column

print(hr_tab4.shape)
hr_tab4.head()

##6,072,003 records

In [ ]:
hr_tab4_patlist = list(hr_tab4['patientid'].unique())
len(hr_tab4_patlist)

##3391 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab4

temp = hr_tab4.groupby('patientid')
dtime_tab4 = temp.agg(max_date=('datetime', np.max))
dtime_tab4 = dtime_tab4.reset_index()
dtime_tab4

#max_date = discharge time/ death time for all patientids in hr_tab4

In [ ]:
#Connect to observations_table_5 - find all HR readings

hr_tab5 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_5 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab5['row_idx']=hr_tab5.index #create new index column

print(hr_tab5.shape)
hr_tab5.head()

##6,059,325 records

In [ ]:
hr_tab5_patlist = list(hr_tab5['patientid'].unique())
len(hr_tab5_patlist)

##3388 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab5

temp = hr_tab5.groupby('patientid')
dtime_tab5 = temp.agg(max_date=('datetime', np.max))
dtime_tab5 = dtime_tab5.reset_index()
dtime_tab5

#max_date = discharge time/ death time for all patientids in hr_tab5

In [ ]:
#Connect to observations_table_6 - find all HR readings

hr_tab6 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_6 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab6['row_idx']=hr_tab6.index #create new index column

print(hr_tab6.shape)
hr_tab6.head()

##6,098,969 records

In [ ]:
hr_tab6_patlist = list(hr_tab6['patientid'].unique())
len(hr_tab6_patlist)

##3391 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab6

temp = hr_tab6.groupby('patientid')
dtime_tab6 = temp.agg(max_date=('datetime', np.max))
dtime_tab6 = dtime_tab6.reset_index()
dtime_tab6

#max_date = discharge time/ death time for all patientids in hr_tab6

In [ ]:
#Connect to observations_table_7 - find all HR readings

hr_tab7 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_7 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab7['row_idx']=hr_tab7.index #create new index column

print(hr_tab7.shape)
hr_tab7.head()

##6,126,086 records

In [ ]:
hr_tab7_patlist = list(hr_tab7['patientid'].unique())
len(hr_tab7_patlist)

##3390 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab7

temp = hr_tab7.groupby('patientid')
dtime_tab7 = temp.agg(max_date=('datetime', np.max))
dtime_tab7 = dtime_tab7.reset_index()
dtime_tab7

#max_date = discharge time/ death time for all patientids in hr_tab7

In [ ]:
#Connect to observations_table_8 - find all HR readings 

hr_tab8 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_8 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab8['row_idx']=hr_tab8.index #create new index column

print(hr_tab8.shape)
hr_tab8.head()

##5,964,420 records

In [ ]:
hr_tab8_patlist = list(hr_tab8['patientid'].unique())
len(hr_tab8_patlist)

##3389 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab8

temp = hr_tab8.groupby('patientid')
dtime_tab8 = temp.agg(max_date=('datetime', np.max))
dtime_tab8 = dtime_tab8.reset_index()
dtime_tab8

#max_date = discharge time/ death time for all patientids in hr_tab8

In [ ]:
#Connect to observations_table_9 - find all HR readings

hr_tab9 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_9 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab9['row_idx']=hr_tab9.index #create new index column

print(hr_tab9.shape)
hr_tab9.head()

##6,163,893 records

In [ ]:
hr_tab9_patlist = list(hr_tab9['patientid'].unique())
len(hr_tab9_patlist)

##3389 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab9

temp = hr_tab9.groupby('patientid')
dtime_tab9 = temp.agg(max_date=('datetime', np.max))
dtime_tab9 = dtime_tab9.reset_index()
dtime_tab9

#max_date = discharge time/ death time for all patientids in hr_tab9

In [ ]:
#Connect to observations_table_10 - find all HR readings

hr_tab10 = pd.read_sql_query(" SELECT * FROM hirid.observations_table_10 \
                                    WHERE variableid IN (200) \
                                    ORDER BY patientid, datetime ", conn)

hr_tab10['row_idx']=hr_tab10.index #create new index column

print(hr_tab10.shape)
hr_tab10.head()

##5,770,397 records

In [ ]:
hr_tab10_patlist = list(hr_tab10['patientid'].unique())
len(hr_tab10_patlist)

##3392 unique admissions

In [ ]:
#Find date of latest Heart Rate for each patientid in hr_tab10

temp = hr_tab10.groupby('patientid')
dtime_tab10 = temp.agg(max_date=('datetime', np.max))
dtime_tab10 = dtime_tab10.reset_index()
dtime_tab10

#max_date = discharge time/ death time for all patientids in hr_tab10

In [ ]:
#Confirm no overlapping patientids across the 10 observations tables

pat_in_all = list(set.intersection(*map(set, [hr_tab1_patlist, hr_tab2_patlist, hr_tab3_patlist, hr_tab4_patlist, hr_tab5_patlist, hr_tab6_patlist,hr_tab7_patlist, hr_tab8_patlist, hr_tab9_patlist, hr_tab10_patlist ])))
pat_in_all

#no overlapping patientids across hr_tab1, ht_tab2, hr_tab3, hr_tab4, ht_tab5, hr_tab6, hr_tab7, ht_tab8, hr_tab9 and hr_tab10

In [ ]:
#Merge above dtime tables to create table of discharge/death times per patientid 

frames = [dtime_tab1, dtime_tab2, dtime_tab3, dtime_tab4, dtime_tab5,
         dtime_tab6, dtime_tab7, dtime_tab8, dtime_tab9, dtime_tab10]

dtimes = pd.concat(frames)

dtimes.sort_values(['patientid'], ascending=[True], inplace=True)

print(dtimes.shape)
dtimes

##33,897 unique patientids (ICU admissions)

In [ ]:
#Connect to HiRID pharma_records table

pat = pd.read_sql_query("SELECT * FROM hirid.patient \
                         ORDER BY patientid", conn)

print(pat.shape)
pat.head()

##33,905 unique ICU admissions 

In [ ]:
#Check unique ICU admissions in 'General' table

pat['patientid'].nunique()

##33,905 unique ICU admissions 

In [ ]:
#Merge dtimes with general_table 

pat_info = pd.merge(pat, dtimes,  how='left', left_on=['patientid'], right_on = ['patientid'])
pat_info.rename({'max_date': 'd_time'}, axis=1, inplace=True)

print(pat_info.shape)
pat_info.head()

In [ ]:
#Create ICU LOS column

pat_info['icu_los'] = pat_info['d_time'] - pat_info['admissiontime']

#convert to hours
pat_info['icu_los'] = (pat_info['icu_los'] / np.timedelta64(1, 'h'))
pat_info.rename({'icu_los': 'icu_los(hrs)'}, axis=1, inplace=True)

print(pat_info.shape)
pat_info.head()

#33,905 records 

In [ ]:
#Create fields for last hours before discharge/death (1hr/2hrs/3hrs/4hrs/7hrs/8hrs)

from datetime import timedelta

pat_info['1hr before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=1)
pat_info['2hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=2)
pat_info['3hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=3)
pat_info['4hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=4)
pat_info['7hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=7)
pat_info['8hrs before d_time'] = pd.to_datetime(pat_info['d_time'])- timedelta(hours=8)

print(pat_info.shape)
pat_info

### 13) Explore patient characteristics of final dataset

In [ ]:
print(comp_avg_params.shape)
comp_avg_params

##2,022,310 records

In [ ]:
#Extract patientids within comp_avg_params

pat_rel = comp_avg_params['patientid'].unique()
pat_rel = pd.DataFrame(data=pat_rel)
pat_rel.columns = ['patientid']

print(pat_rel.shape)
pat_rel

#20,073 records

In [ ]:
print(pat_info.shape)
pat_info

##33,905 unique ICU admissions

In [ ]:
#Merge pat_rel and pat_info

df_patmerge = pd.merge(pat_rel, pat_info, how='left', left_on=['patientid'], right_on=['patientid'])
print(df_patmerge.shape)
df_patmerge.head()

##20,073 records

In [ ]:
df_patmerge.describe()

##mean age=64.3
##mean ICU LOS = 69.1 hours

In [ ]:
df_patmerge['sex'].value_counts()

##13,659 males
##6414 females

In [ ]:
df_patmerge['discharge_status'].value_counts()

##17,919 discharged alive
##1,951 discharged dead

In [ ]:
#Check how many of the 33,905 unique ICU admissions were adminsitered Adrenaline/Noradrenaline

pat_all = list(pat['patientid'].unique())

len(list(set(pat_all).intersection(pat_drugs)))

## 14) Create HiRID External Validation Dataset

In [ ]:
#Define 1hr before discharge/death time period

dtime_pre1hr = pat_info.copy(deep=True)
dtime_pre1hr = dtime_pre1hr[['patientid', 'd_time', '1hr before d_time']]

print(dtime_pre1hr.shape)
dtime_pre1hr

##33,905 records

In [ ]:
#Filter for only instances with readings in the last hour before discharge/death

params_1hr = pd.merge(comp_avg_params, dtime_pre1hr, how='left', left_on=['patientid'], right_on = ['patientid'])
params_1hr = params_1hr[(params_1hr['timepoint']<params_1hr['d_time']) & (params_1hr['timepoint']>params_1hr['1hr before d_time'])]

params_1hr.to_csv("params1hr.csv")

print(params_1hr.shape)
params_1hr

##3117 records

In [ ]:
#Import HiRID 'General' table (contains discharge_status info)
#General table (& other HiRID tables) are found here: https://www.physionet.org/content/hirid/1.1.1/

pat = pd.read_csv("./general_table.csv")
print(pat.shape)
pat.head()

In [ ]:
#Add discharge status columns to params1hr dataframe

params1hr['Time before dis/death(hrs)'] = 1
params1hr = pd.merge(params1hr, pat, how='left', left_on=['patientid'], right_on = ['patientid'])

print(params1hr.shape)
params1hr.head()

In [ ]:
#Drop all rows containing null values

params1hr = params1hr.dropna()

print(params1hr.shape)
params1hr.head()

In [ ]:
#Create Final HiRID External Validation Dataset (balanecd on discharge status classes)

sub_zero = test_zero.sample(n=1300, random_state=1)
sub_one = test_one.sample(n=1300, random_state=1)

frames = [sub_zero, sub_one]

t_params1hr = pd.concat(frames)

t_params1hr = t_params1hr.sort_values(by='rowID', ascending=True)
t_params1hr = t_params1hr.reset_index()
t_params1hr = t_params1hr.drop('index', axis=1)

t_params1hr.to_csv('HiRID_extval_params1hr.csv')

print(t_params1hr.shape)
t_params1hr

##t_params1hr = Hirid external validation dataset

In [ ]:
#Check class balance of HiRID External Validation Dataset

t_params1hr['binary_status'].value_counts()